In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.8 MB/s eta 0:00:00


In [2]:
from transformers import TFBertModel,BertConfig,AutoTokenizer

config = BertConfig.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", output_hidden_states=True)
model = TFBertModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", config=config)


checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")["input_ids"]
print(inputs)




Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFBertModel: ['pre_classifier', 'dropout_19', 'classifier', 'distilbert']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

tf.Tensor(
[[  101  1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026
   2878  2166  1012   102]
 [  101  1045  5223  2023  2061  2172   999   102     0     0     0     0
      0     0     0     0]], shape=(2, 16), dtype=int32)


# Extracting Last Layer hidden states

In [6]:
import numpy as np
outputs = model(inputs)


## 0 th index is last hidden layer

In [7]:
print('shape of last hidden layer :',np.array(outputs[0]).shape)  # 3

shape of last hidden layer : (2, 16, 768)


In [8]:
print(' *********** last hidden layer *********** ')
print(np.array(outputs[0]))

 *********** last hidden layer *********** 
[[[-0.6818589  -0.2271215  -1.4330205  ...  0.43062946  1.0059807
   -0.5503813 ]
  [-0.5162936   0.78841084 -1.8106267  ...  1.3160259   1.5768292
    0.62573326]
  [-1.6987885  -0.47626844 -1.9268193  ...  1.2493583   1.6204984
    1.1921031 ]
  ...
  [-1.3341396   0.7941447  -2.9939675  ...  2.285175    1.3894361
   -0.0606142 ]
  [-0.85912645  0.29380956 -1.8461064  ...  0.69391036  0.4082715
    1.5897641 ]
  [-2.9078724  -0.6399442  -1.4704522  ...  1.0265151   2.1297193
    1.101027  ]]

 [[-1.128914   -0.2879271  -1.5258256  ...  0.5840709   0.90408885
   -0.3272105 ]
  [-0.8956347   0.8466187  -1.8474655  ...  1.5345343   1.3904626
    0.610588  ]
  [-2.1321692  -0.6555472  -1.6027515  ...  1.0561491   0.9472406
    0.5281486 ]
  ...
  [-1.2973387   1.9020091  -2.1892722  ...  1.1332862   1.078896
    0.36505175]
  [-1.8555789   1.0616411  -1.8218828  ...  0.6714737   1.0461587
    1.649019  ]
  [-2.8149667   1.2828333  -1.3919083  .

# Extracting sencting enbeddings named as pooled embedding vectors of the sentence

## 1st Index have pooled embeddings defaults strategy is the reduce_mean

In [9]:
sentence_vectors = outputs[1]

In [11]:
print('shape of pooled embeddings :',np.array(sentence_vectors).shape)
print(' *********** pooled embeddings (sentence vectors) *********** ')
print(np.array(sentence_vectors))

shape of pooled embeddings : (2, 768)
 *********** pooled embeddings (sentence vectors) *********** 
[[-0.36821455 -0.6791278  -0.24746285 ...  0.1769942  -0.5956687
   0.44644713]
 [-0.26382324 -0.5884107  -0.18373148 ...  0.07583709 -0.54250836
   0.3261593 ]]


## Extracting all the hiddenstates along with the temporal embedding vectors

## 2nd index have the all hidden layer vectors here we have 13 layers because 12 for attention and 1 for temporal embedding

In [12]:
hidden_states = outputs[2]


In [13]:
print('shape of pooled embeddings :',np.array(hidden_states).shape)

shape of pooled embeddings : (13, 2, 16, 768)


In [14]:
attention_hidden_states = np.array(hidden_states)[1:]
print(attention_hidden_states)

[[[[-8.0008686e-01  1.1047724e+00  6.7548734e-01 ... -4.9741635e-01
    -1.9958380e-01 -4.9878058e-01]
   [ 5.1531992e-03  1.3219559e+00  3.2393488e-01 ... -1.2478192e-01
     3.8913791e-03 -3.2007813e-01]
   [-1.6763496e+00  2.0572660e+00 -3.7808385e-01 ... -9.0358073e-01
     6.3802260e-01  6.1118442e-01]
   ...
   [-1.1934903e+00  3.1602950e+00 -6.1908418e-01 ...  1.3850715e+00
     8.1395453e-01 -8.9025033e-01]
   [ 2.4699491e-01  6.4110726e-01  3.3133349e-01 ... -6.8953800e-01
     8.6571962e-01 -2.5037804e-01]
   [-1.7177323e+00  1.9008249e+00  1.3808062e+00 ...  6.8363476e-01
     1.1843054e+00 -6.4517863e-02]]

  [[-7.7853066e-01  1.1690859e+00  6.5074086e-01 ... -6.1032307e-01
    -2.5285298e-01 -4.5152676e-01]
   [ 3.5528474e-02  1.3923500e+00  2.7173463e-01 ... -2.1958964e-01
    -3.8722277e-02 -2.6562062e-01]
   [-1.6328439e+00  1.9914621e+00  3.2595569e-01 ... -1.1109294e+00
    -1.8086812e-02 -2.3343232e-01]
   ...
   [-9.2231107e-01  3.2627468e+00  1.0825732e-01 ... -5.1